# Liquid Drop Model

Implemenent the liquid drop model formula for the binding energy,

$$ E_{\mathrm{binding,MeV}} = 15.75A -17.8A^{2/3}-.711Z^2A^{-1/3}-23.7\frac{(A-2Z)^2}{A} + 11.18 \delta_{\mathrm{pair}}A^{-1/2} $$

In [123]:
import mpld3
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
mpl.rc("savefig", dpi=300)
%matplotlib inline

def binding_energy(Z,A):
    Z,A = Z.astype(np.int64),A.astype(np.int64)
    d = .5 * (((2*(Z%2))-1) + ((2*(A%2))-1))
    return 15.75*A -17.8*(A**(2./3)) - .711*(Z**2)/(A**(1./3)) -23.7*((A-2*Z)**2)/A + 11.18*(A**(-.5))*(d)

def naive_mass(Z,A):
    return (938.272+.511) * Z + ( 939.565)* (A-Z)

def stable_Z(A):
    return np.rint(A /( 2 + (.5*.711/23.7)*A**(2/3.)))

Try out the liquid drop model on $A=10$. We can see that $Z=5$ (Boron) is the most stable isotope in this case, only 10.17 MeV heavier than 10 amu.

In [71]:
A = np.array([10,10,10,10])
Z = np.array([3,4,5,6])

print (naive_mass(Z,A) - binding_energy(Z,A)) - (931.494*10)

[ 44.37443393  20.99797912  10.17070539  26.03431844]


We can see that this mass excess anitcorrelates with half life:

|Z|  Predicted Mass Excess (MeV) | $T_{1/2}$ (seconds)|
|::|:-----------------:|:------------------------------------------------------------:|
|B |10.17| $4.73 \times 10^{13}$  |
|Be |20.997|$4.38 \times 10^{13}$ |
|C |26.034| $19.29$  |
|Li |44.37| $2\times 10^{-21}$ |


I'm not sure what's up with Be-10. It looks like it eventually decays to B-10 via a .556 MeV electron, but it's not clear why that particular table doesn't list a mass for it. It's definitely long lived enough to measure easily, and doesn't seem to be exceptionally rare. 

## Binding Energy Per Nucleon of Stable Isotopes

In [124]:
with open('binding.txt','r') as f:
    lines = f.readlines()
    
A,b = [],[]

for line in lines:
    if '#' in line or 'X' in line:
        continue
        
    line = line.split(' ')
    
    A.append(line[0]) 
    b.append(line[5]) # binding energy per nucleon
    
A = np.array(A,dtype=np.float)
b = np.array(b,dtype=np.float)

# break these arrays into groups of common A

last_a = A[0]
running_min = b[0]

idx = []
i0,j0 = 0,0

for i,a in enumerate(A):
    if a != last_a:
        j0 = i
        idx.append(slice(i0,j0))
        i0 = j0
    last_a = a

# Find the minimum binding energy per nucleon at a given A
    
A = np.array([A[s][0] for s in idx])
b_stable_published = np.array([np.min(b[s]) for s in idx])

# Find the liquid drop model prediction at every A

b_model = binding_energy(stable_Z(A),A)/A


Plot the binding energy per nucleon for stable isotopoes against mass number. Liquid drop model predictions are shown in blue, published values in green.

In [131]:
plt.figure(figsize=(12,7))
plt.plot(A,b_model)
plt.plot(A,b_stable_published)
plt.xlabel('A')
plt.ylabel('Binding Energy Per Nucleon MeV')
mpld3.display()